## Finding the most populous zip code in each county

In [2]:
rm(list=ls())
libs <- c("dplyr","openxlsx","tidyr","writexl")
load.libs <- function(x){
  for(i in x){
    if(!require(i,character.only = TRUE )){
      install.packages(i,dependencies = TRUE )
      library(i,character.only = TRUE ) 
    }
  }
}
suppressWarnings(load.libs(libs))

Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: openxlsx
Loading required package: tidyr
Loading required package: writexl


In [3]:
#data set that includes every zip code in the US + its county as a county number

zips <- read.xlsx("C:/Users/asyah/Desktop/Fall 2020/Medicaid/ZIP_COUNTY_122018.xlsx")
head(zips)

zip,county,res_ratio,bus_ratio,oth_ratio,tot_ratio
00616,72013,1.00000000,1.0000000,1,1.00000000
00617,72054,0.01677129,0.0000000,0,0.01567563
00624,72111,0.89446017,0.8307692,1,0.89176441
00652,72013,1.00000000,1.0000000,1,1.00000000
00660,72067,0.99615498,1.0000000,1,0.99635548
00670,72083,1.00000000,1.0000000,1,1.00000000


In [4]:
#only keep the zip code that contains the highest percentage of residential addresses in the county

maxzips <- zips %>% group_by(county) %>% slice(which.max(res_ratio))

In [5]:
head(maxzips)
summary(maxzips)
length(unique(maxzips$county))

zip,county,res_ratio,bus_ratio,oth_ratio,tot_ratio
36003,01001,1.0000000,1.0000000,1,1.0000000
36550,01003,1.0000000,1.0000000,0,1.0000000
36017,01005,1.0000000,1.0000000,1,1.0000000
35035,01007,0.9796163,0.9565217,1,0.9791183
35097,01009,1.0000000,1.0000000,1,1.0000000
36089,01011,0.9129911,0.9822335,1,0.9197733


     zip               county            res_ratio        bus_ratio     
 Length:3225        Length:3225        Min.   :0.0000   Min.   :0.0000  
 Class :character   Class :character   1st Qu.:1.0000   1st Qu.:1.0000  
 Mode  :character   Mode  :character   Median :1.0000   Median :1.0000  
                                       Mean   :0.9906   Mean   :0.9248  
                                       3rd Qu.:1.0000   3rd Qu.:1.0000  
                                       Max.   :1.0000   Max.   :1.0000  
   oth_ratio        tot_ratio        
 Min.   :0.0000   Min.   :0.0004174  
 1st Qu.:1.0000   1st Qu.:1.0000000  
 Median :1.0000   Median :1.0000000  
 Mean   :0.9088   Mean   :0.9948696  
 3rd Qu.:1.0000   3rd Qu.:1.0000000  
 Max.   :1.0000   Max.   :1.0000000  

[1] 3225

In [6]:
#countains county names and their codes
# HUD USPS Zipcode Crosswalk

codes <- read.xlsx("C:/Users/asyah/Desktop/Fall 2020/Medicaid/all-geocodes-v2018.xlsx", startRow= 5)

In [10]:
#select the state codes (to be used to merge state name)

states <- codes %>% filter(Summary.Level == "040") %>% select("State.Code", "Area.Name") 

In [8]:
#select county numbers
countynums <- codes %>% filter(Summary.Level == "050") %>% 

#pasting the strings for state code and county number to give full county code
select("State.Code","County.Code","Area.Name") %>% unite(county, State.Code, County.Code, sep="",remove = FALSE)

nrow(countynums)

[1] 3220

In [11]:
zipcounty <- (merge(countynums, maxzips, by = 'county'))

zipcounty2 <- (merge(states, zipcounty, by = "State.Code"))

head(zipcounty2)

State.Code,Area.Name.x,county,County.Code,Area.Name.y,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
01,Alabama,01001,001,Autauga County,36003,1.0000000,1.0000000,1,1.0000000
01,Alabama,01003,003,Baldwin County,36550,1.0000000,1.0000000,0,1.0000000
01,Alabama,01005,005,Barbour County,36017,1.0000000,1.0000000,1,1.0000000
01,Alabama,01007,007,Bibb County,35035,0.9796163,0.9565217,1,0.9791183
01,Alabama,01009,009,Blount County,35097,1.0000000,1.0000000,1,1.0000000
01,Alabama,01011,011,Bullock County,36089,0.9129911,0.9822335,1,0.9197733


In [14]:
#read in names of Medicaid FFS for each state 

mcaid <- read.xlsx("C:/Users/asyah/Desktop/Fall 2020/Medicaid/mcaidnames.xlsx")

#merge by state code
zipcounty3 <- (merge(select(mcaid, "State.Code","Medicaid.FFS.Listed.on.MinuteClinic.Dropdown"), 
                     zipcounty2, by = "State.Code"))

#remove states that do not have CVS MinuteClinics
zipmed <- zipcounty3 %>% select("Medicaid.FFS.Listed.on.MinuteClinic.Dropdown","zip") %>% 
filter(zipcounty3$Medicaid != "NONE")

str(zipmed)

'data.frame':	1756 obs. of  2 variables:
 $ Medicaid.FFS.Listed.on.MinuteClinic.Dropdown: chr  "CT MEDICAID" "CT MEDICAID" "CT MEDICAID" "CT MEDICAID" ...
 $ zip                                         : chr  "06484" "06040" "06759" "06457" ...
